In [9]:
%load_ext nb_black
%matplotlib widget
import copy
import datetime
import json
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from importlib import reload

import celeri
celeri = reload(celeri)

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [10]:
RUN_NAME = datetime.datetime.now().strftime("%y%m%d%H%M%S") + os.sep
with open('./data/western_north_america/command.json', 'r') as f:
    command = json.load(f)
station = pd.read_csv(command["station_file_name"])
segment = pd.read_csv(command["segment_file_name"])
block = pd.read_csv(command["block_file_name"])

<IPython.core.display.Javascript object>

In [11]:
station = celeri.process_station(station, command)
segment = celeri.process_segment(segment, command)

<IPython.core.display.Javascript object>

In [12]:
np_segments = np.zeros((len(segment), 2, 2))
np_segments[:, 0, 0] = segment.lon1.to_numpy()
np_segments[:, 1, 0] = segment.lon2.to_numpy()
np_segments[:, 0, 1] = segment.lat1.to_numpy()
np_segments[:, 1, 1] = segment.lat2.to_numpy()
for i in range(np_segments.shape[0] - 1):
    plt.plot(np_segments[i,:,0], np_segments[i,:,1], 'k-')
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<IPython.core.display.Javascript object>